In [39]:
import pandas as pd
import numpy as np

In [40]:
from catboost import CatBoostRegressor

In [41]:
from catboost import Pool

In [42]:
from sklearn.metrics import mean_squared_error

In [43]:
loc_lbl = r"D:\TP2_Machine_Learning_v4\Labels"

In [68]:
loc_ftr = r"D:\TP2_Machine_Learning_v4\Features\FeaturesST"

In [45]:
loc_tr = r"D:\TP2_Machine_Learning\Training Sets"

In [46]:
auc_lb = pd.read_csv( loc_lbl + "\\auc_lb.csv", dtype = {"obs_18_20":"bool", "obs_21_23":"bool", "obs_24_26":"bool"} )

### Filtro de manera que tenga los datos necesarios

In [47]:
rh_18_20 = pd.read_csv( loc_tr + "\\rh_18_20_st.csv" ).rename( columns = {"device_id":"ref_hash"} ); rh_18_20 = rh_18_20["ref_hash"].drop_duplicates().tolist()
rh_21_23 = pd.read_csv( loc_tr + "\\rh_21_23_st.csv" ).rename( columns = {"device_id":"ref_hash"} ); rh_21_23 = rh_21_23["ref_hash"].drop_duplicates().tolist()

In [50]:
dt_18_20 = auc_lb.loc[ auc_lb["ref_hash"].isin( rh_18_20 ), ["ref_hash"] ] #Datos del (18-20)
st_21_23 = auc_lb.loc[ auc_lb["ref_hash"].isin( rh_18_20 ), ["21_23_st"] ] #SC del (21-23)

dt_21_23 = auc_lb.loc[ auc_lb["ref_hash"].isin( rh_21_23 ), ["ref_hash"] ] #Datos del (21-23)
st_24_26 = auc_lb.loc[ auc_lb["ref_hash"].isin( rh_21_23 ), ["24_26_st"] ] #SC del (24-26)

# <span style="color:orange">  Preparo los datos para predecir </span>
Con los datos 21-23 predecimos "24-26_sc"

In [51]:
set_trn = auc_lb.loc[ auc_lb["ref_hash"].isin( rh_18_20 ), ["ref_hash", "21_23_st"] ]
set_tst = auc_lb.loc[ auc_lb["ref_hash"].isin( rh_21_23 ), ["ref_hash", "24_26_st"] ]

t_lim = 259200.00
max_t = 200

set_trn_loc = pd.concat( [ set_trn.loc[ set_trn["21_23_st"] < t_lim ], set_trn.loc[ set_trn["21_23_st"] == t_lim ].reset_index( drop = True ).loc[: max_t] ], axis = 0 ).sample(frac=1).reset_index(drop=True) 
set_tst_loc = pd.concat( [ set_tst.loc[ set_tst["24_26_st"] < t_lim ], set_tst.loc[ set_tst["24_26_st"] == t_lim ].reset_index( drop = True ).loc[: max_t] ], axis = 0 ).sample(frac=1).reset_index(drop=True) 

In [52]:
X_loc = set_trn_loc.loc[:,['ref_hash']] #Datos train
Y_loc = set_trn_loc.loc[:,['21_23_st']] #Label train

Z_loc = set_tst_loc.loc[:,['ref_hash']] #Datos test
W_loc = set_tst_loc.loc[:,['24_26_st']] #Label test

In [69]:
X = X_loc
Y = Y_loc
Z = Z_loc
W = W_loc

### Agregamos los features 

In [ ]:
#ftr_01_Z = pd.read_csv( loc_ftr + "\\hora_auc_21_23.csv" ) 
ftr_02_Z = pd.read_csv( loc_ftr + "\\cant_clk_21_23.csv" ) 
ftr_03_Z = pd.read_csv( loc_ftr + "\\cant_ins_21_23.csv" ) 
ftr_04_Z = pd.read_csv( loc_ftr + "\\cant_auc_21_23.csv" ) 
ftr_05_Z = pd.read_csv( loc_ftr + "\\cant_evt_21_23.csv" )
#ftr_06_Z = pd.read_csv( loc_ftr + "\\ref_type_21_23.csv" )  
ftr_07_Z = pd.read_csv( loc_ftr + "\\sdia_auc_21_23.csv" )
#ftr_08_Z = pd.read_csv( loc_ftr + "\\srce_auc_21_23.csv" )
ftr_09_Z = pd.read_csv( loc_ftr + "\\rh_encod_21_23.csv" )
ftr_10_Z = pd.read_csv( loc_ftr + "\\main_ahr_21_23.csv" )
ftr_11_Z = pd.read_csv( loc_ftr + "\\kind_evt_21_23.csv" )
ftr_12_Z = pd.read_csv( loc_ftr + "\\frst_auc_21_23.csv" )

ftr_13_Z = pd.read_csv( loc_ftr + "\\frst_evt_21_23.csv" )
ftr_14_Z = pd.read_csv( loc_ftr + "\\frst_ins_21_23.csv" )
ftr_15_Z = pd.read_csv( loc_ftr + "\\frst_clk_21_23.csv" )

ftr_16_Z = pd.read_csv( loc_ftr + "\\last_auc_21_23.csv" )
ftr_17_Z = pd.read_csv( loc_ftr + "\\last_evt_21_23.csv" )

ftr_18_Z = pd.read_csv( loc_ftr + "\\me_mt_1_auc_21_23.csv" )
ftr_19_Z = pd.read_csv( loc_ftr + "\\mt_1_auc_21_23.csv" )

ftr_20_Z = pd.read_csv( loc_ftr + "\\me_auc_night_21_23.csv" )
#ftr_21_Z = pd.read_csv( loc_ftr + "\\auc_night_21_23.csv" )
ftr_22_Z = pd.read_csv( loc_ftr + "\\me_auc_morn_21_23.csv" )
#ftr_23_Z = pd.read_csv( loc_ftr + "\\auc_morn_21_23.csv" )
ftr_24_Z = pd.read_csv( loc_ftr + "\\me_auc_midday_21_23.csv" )
#ftr_25_Z = pd.read_csv( loc_ftr + "\\auc_midday_21_23.csv" )
ftr_26_Z = pd.read_csv( loc_ftr + "\\me_auc_after_21_23.csv" )
#ftr_27_Z = pd.read_csv( loc_ftr + "\\auc_after_21_23.csv" )

In [ ]:
#Z = Z.merge( ftr_01_Z, how = "inner", on = "ref_hash" )
Z = Z.merge( ftr_02_Z, how = "inner", on = "ref_hash" )
Z = Z.merge( ftr_03_Z, how = "inner", on = "ref_hash" )
Z = Z.merge( ftr_04_Z, how = "inner", on = "ref_hash" )
Z = Z.merge( ftr_05_Z, how = "inner", on = "ref_hash" )
#Z = Z.merge( ftr_06_Z, how = "inner", on = "ref_hash" )
Z = Z.merge( ftr_07_Z, how = "inner", on = "ref_hash" )
#Z = Z.merge( ftr_08_Z, how = "inner", on = "ref_hash" )
Z = Z.merge( ftr_09_Z, how = "inner", on = "ref_hash" )
Z = Z.merge( ftr_10_Z, how = "inner", on = "ref_hash" )
Z = Z.merge( ftr_11_Z, how = "inner", on = "ref_hash" )
Z = Z.merge( ftr_12_Z, how = "inner", on = "ref_hash" )

Z = Z.merge( ftr_13_Z, how = "inner", on = "ref_hash" )
Z = Z.merge( ftr_14_Z, how = "inner", on = "ref_hash" )
Z = Z.merge( ftr_15_Z, how = "inner", on = "ref_hash" )

Z = Z.merge( ftr_16_Z, how = "inner", on = "ref_hash" )
Z = Z.merge( ftr_17_Z, how = "inner", on = "ref_hash" )

Z = Z.merge( ftr_18_Z, how = "inner", on = "ref_hash" )
Z = Z.merge( ftr_19_Z, how = "inner", on = "ref_hash" )

Z = Z.merge( ftr_20_Z, how = "inner", on = "ref_hash" )
#Z = Z.merge( ftr_21_Z, how = "inner", on = "ref_hash" )
Z = Z.merge( ftr_22_Z, how = "inner", on = "ref_hash" )
#Z = Z.merge( ftr_23_Z, how = "inner", on = "ref_hash" )
Z = Z.merge( ftr_24_Z, how = "inner", on = "ref_hash" )
#Z = Z.merge( ftr_25_Z, how = "inner", on = "ref_hash" )
Z = Z.merge( ftr_26_Z, how = "inner", on = "ref_hash" )
#Z = Z.merge( ftr_27_Z, how = "inner", on = "ref_hash" )

# <span style="color:orange">  Preparo los datos para entrenar </span>

Con los datos 18-20 + "21-23_sc" entrenamos

### Agregamos los features

In [ ]:
#ftr_01_X = pd.read_csv( loc_ftr + "\\hora_auc_18_20.csv" ) 
ftr_02_X = pd.read_csv( loc_ftr + "\\cant_clk_18_20.csv" ) 
ftr_03_X = pd.read_csv( loc_ftr + "\\cant_ins_18_20.csv" ) 
ftr_04_X = pd.read_csv( loc_ftr + "\\cant_auc_18_20.csv" ) 
ftr_05_X = pd.read_csv( loc_ftr + "\\cant_evt_18_20.csv" )
#ftr_06_X = pd.read_csv( loc_ftr + "\\ref_type_18_20.csv" )  
ftr_07_X = pd.read_csv( loc_ftr + "\\sdia_auc_18_20.csv" )  
#ftr_08_X = pd.read_csv( loc_ftr + "\\srce_auc_18_20.csv" )
ftr_09_X = pd.read_csv( loc_ftr + "\\rh_encod_18_20.csv" )
ftr_10_X = pd.read_csv( loc_ftr + "\\main_ahr_18_20.csv" )
ftr_11_X = pd.read_csv( loc_ftr + "\\kind_evt_18_20.csv" )
ftr_12_X = pd.read_csv( loc_ftr + "\\frst_auc_18_20.csv" )

ftr_13_X = pd.read_csv( loc_ftr + "\\frst_evt_18_20.csv" )
ftr_14_X = pd.read_csv( loc_ftr + "\\frst_ins_18_20.csv" )
ftr_15_X = pd.read_csv( loc_ftr + "\\frst_clk_18_20.csv" )

ftr_16_X = pd.read_csv( loc_ftr + "\\last_auc_18_20.csv" )
ftr_17_X = pd.read_csv( loc_ftr + "\\last_evt_18_20.csv" )

ftr_18_X = pd.read_csv( loc_ftr + "\\me_mt_1_auc_18_20.csv" )
ftr_19_X = pd.read_csv( loc_ftr + "\\mt_1_auc_18_20.csv" )

ftr_20_X = pd.read_csv( loc_ftr + "\\me_auc_night_18_20.csv" )
#ftr_21_X = pd.read_csv( loc_ftr + "\\auc_night_18_20.csv" )
ftr_22_X = pd.read_csv( loc_ftr + "\\me_auc_morn_18_20.csv" )
#ftr_23_X = pd.read_csv( loc_ftr + "\\auc_morn_18_20.csv" )
ftr_24_X = pd.read_csv( loc_ftr + "\\me_auc_midday_18_20.csv" )
#ftr_25_X = pd.read_csv( loc_ftr + "\\auc_midday_18_20.csv" )
ftr_26_X = pd.read_csv( loc_ftr + "\\me_auc_after_18_20.csv" )
#ftr_27_X = pd.read_csv( loc_ftr + "\\auc_after_18_20.csv" )

In [ ]:
#X = X.merge( ftr_01_X, how = "inner", on = "ref_hash" )
X = X.merge( ftr_02_X, how = "inner", on = "ref_hash" )
X = X.merge( ftr_03_X, how = "inner", on = "ref_hash" )
X = X.merge( ftr_04_X, how = "inner", on = "ref_hash" )
X = X.merge( ftr_05_X, how = "inner", on = "ref_hash" )
#X = X.merge( ftr_06_X, how = "inner", on = "ref_hash" )
X = X.merge( ftr_07_X, how = "inner", on = "ref_hash" )
#X = X.merge( ftr_08_X, how = "inner", on = "ref_hash" )
X = X.merge( ftr_09_X, how = "inner", on = "ref_hash" )
X = X.merge( ftr_10_X, how = "inner", on = "ref_hash" )
X = X.merge( ftr_11_X, how = "inner", on = "ref_hash" )
X = X.merge( ftr_12_X, how = "inner", on = "ref_hash" )

X = X.merge( ftr_13_X, how = "inner", on = "ref_hash" )
X = X.merge( ftr_14_X, how = "inner", on = "ref_hash" )
X = X.merge( ftr_15_X, how = "inner", on = "ref_hash" )

X = X.merge( ftr_16_X, how = "inner", on = "ref_hash" )
X = X.merge( ftr_17_X, how = "inner", on = "ref_hash" )

X = X.merge( ftr_18_X, how = "inner", on = "ref_hash" )
X = X.merge( ftr_19_X, how = "inner", on = "ref_hash" )

X = X.merge( ftr_20_X, how = "inner", on = "ref_hash" )
#X = X.merge( ftr_21_X, how = "inner", on = "ref_hash" )
X = X.merge( ftr_22_X, how = "inner", on = "ref_hash" )
#X = X.merge( ftr_23_X, how = "inner", on = "ref_hash" )
X = X.merge( ftr_24_X, how = "inner", on = "ref_hash" )
#X = X.merge( ftr_25_X, how = "inner", on = "ref_hash" )
X = X.merge( ftr_26_X, how = "inner", on = "ref_hash" )
#X = X.merge( ftr_27_X, how = "inner", on = "ref_hash" )

## Dropeamos los ref_hash (Quedan los codificados del feature)

In [ ]:
X = X.drop( "ref_hash", axis = 1 )
Z = Z.drop( "ref_hash", axis = 1 )

# Prediccion con <span style="color:green"> *CATBOOST*</span> 

In [ ]:
#Initialize CatBoostRegressor
model = CatBoostRegressor(iterations=200,
                          learning_rate=0.06,
                          depth=3,
                          loss_function='RMSE')

In [ ]:
# Fit model
model.fit(X, Y, verbose=False)

In [ ]:
preds = model.predict(Z)

### Calculamos el RMSE

In [ ]:
rmse = np.sqrt( mean_squared_error(W, preds) )
print("RMSE: %f" % (rmse) )

In [ ]:
record = 63018.200114

## GridSearch CV

### X: Train Data, Y: Train Labels, Z: Test Data, sc_24_26: Test Labels

### Record = iterations=300, learning_rate=0.06, depth=3, RMSE: RMSE: 89111.002876

from sklearn.model_selection import GridSearchCV

In [19]:
#parameters = {'depth'         : [2,3,4,6,8],
#              'learning_rate' : [0.02, 0.05, 0.06, 0.1, 0.5],
#              'iterations'    : [10, 30, 50, 100, 200]
#                 }

from sklearn.metrics import mean_squared_error, make_scorer
mse = make_scorer( mean_squared_error, greater_is_better = False ) 

model = CatBoostRegressor(loss_function='RMSE')
parameters = {'depth'         : [2,3,6,8],
              'learning_rate' : [0.04, 0.06, 0.08, 0.1],
              'iterations'    : [20, 50, 100, 150]
                 }
grid = GridSearchCV(estimator=model, scoring=mse ,param_grid = parameters, cv = 8, n_jobs=2)
grid.fit(X, Y)    

    # Results from Grid Search
    print("\n========================================================")
    print(" Results from Grid Search " )
    print("========================================================")    
    
    print("\n The best estimator across ALL searched params:\n",
          grid.best_estimator_)
    
    print("\n The best score across ALL searched params:\n",
          grid.best_score_)
    
    print("\n The best parameters across ALL searched params:\n",
          grid.best_params_)
    
    print("\n ========================================================")

### Parametros encontrados con grid-searchCV